In [1]:
import os
import pandas as pd
from collections import Counter, defaultdict
import dateparser
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,
    DatesExtractor,
    MoneyExtractor,
    AddrExtractor,

    Doc
)

In [2]:
path = '/home/michael/Documents/Python_practice/vadim_course/Parse/articles_2019-2020/articles/'
files = os.listdir(path)

In [3]:
df1= pd.read_csv(path + files[0], names = \
                 ['Link', 'Headline', 'Date', 'Tags', 'Number of comments', 'Author', 'Text'])
df2 = pd.read_csv(path + files[1])
df3 = pd.read_csv(path + files[2])

In [4]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28249 entries, 0 to 28248
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Link                28249 non-null  object
 1   Headline            28249 non-null  object
 2   Date                28249 non-null  object
 3   Tags                28249 non-null  object
 4   Number of comments  28249 non-null  int64 
 5   Author              28249 non-null  object
 6   Text                28249 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.5+ MB


In [ ]:
from ast import literal_eval
df.Tags =  [literal_eval(row_tag) for row_tag in df.Tags]

In [30]:
df['Date'] = df.Date.apply(lambda x: dateparser.parse(x, languages=['ru'], date_formats=['%d %B %H %M']))

In [20]:
article = df.iloc[0].Text

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

segmenter = Segmenter() #обёртка Razdel
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

doc_article = Doc(article)

doc_article.segment(segmenter)
doc_article.tag_morph(morph_tagger)
doc_article.parse_syntax(syntax_parser)
doc_article.tag_ner(ner_tagger)


In [71]:
def ner_article(x):
    article = x
    doc_article = Doc(article)

    doc_article.segment(segmenter)
    doc_article.tag_morph(morph_tagger)
    doc_article.parse_syntax(syntax_parser)
    doc_article.tag_ner(ner_tagger)
    ner = defaultdict(list)
    for ne in doc_article.spans:
        ner[ne.type].append(ne.text)
    return dict(ner)


In [73]:
df['Named_entity'] = df.Text.apply(lambda x: ner_article(x) )

In [72]:
ner_article(df.iloc[0].Text)

{'ORG': ['UFC', 'Instagram', 'Parimatch Russia', 'UFC', 'UFC'],
 'PER': ['Конор Макгрегор',
  'Париматч',
  'Хабиба Нурмагомедова',
  'Джастином Гейджи',
  'Хабиб',
  'Гейджи',
  'Хабиба',
  'Хабиб',
  'Гейджи',
  'Гейджи',
  'Хабиб',
  'Хабиб',
  'Гейджи',
  'Макгрегор',
  'Гейджи',
  'Джикия',
  'Хабибу',
  'Хабиба',
  'Гейджи',
  'Хабиба Нурмагомедова',
  'Хабиб',
  'Гейджи'],
 'LOC': ['Абу-Даби']}

In [74]:
df.head()

,Link,Headline,Date,Tags,Number of comments,Author,Text,Named_entity
0,https://www.sports.ru/boxing/1090274326-konor-...,"Конор: «Хабиб скорее всего победит, но если Г...",2020-10-24 12:46:00,"['UFC', 'MMA', 'легкий вес (MMA)', 'Хабиб Нурм...",62,Даша Смит,Бывший чемпион UFC в двух весовых категориях К...,"{'ORG': ['UFC', 'Instagram', 'Parimatch Russia..."
1,https://www.sports.ru/kazakhstan/1090274341-da...,Даиров покинет «Кайрат». Его не устраивает от...,2020-10-24 12:24:00,"['Кайрат', 'Окжетпес', 'Нурлан Даиров', 'Алекс...",0,Сейл Кожатай,25-летний защитник «Кайрата» Нурлан Даиров нам...,"{'ORG': ['Кайрата', 'Жетысу', 'КПЛ', 'Окжетпес..."
2,https://www.sports.ru/tennis/1090274384-arina-...,Арина Соболенко: «Мне просто хотелось взять х...,2020-10-24 13:02:00,"['WTA', 'Сара Соррибес-Тормо', 'Арина Соболенк...",4,Tennisonoff,12-я ракетка мира Арина Соболенко прокомментир...,"{'PER': ['Арина Соболенко', 'Сарой Соррибес-То..."
3,https://www.sports.ru/hockey/1090274437-andrej...,Андрей Разин: «Завести соцсети сподвигло то с...,2020-10-24 12:35:00,"['Северсталь', 'болельщики', 'КХЛ', 'интернет'...",0,Александр Балабанов,Главный тренер «Северстали» Андрей Разин проко...,"{'ORG': ['Северстали', 'Instagram'], 'PER': ['..."
4,https://www.sports.ru/basketball/1090274452-ra...,"Райли намекнул, что «Майами» хочет сохранить ...",2020-10-24 12:48:00,"['Майами', 'Пэт Райли', 'НБА', 'Горан Драгич',...",10,Валерий Левкин,Президент «Майами» Пэт Райли во время общения ...,"{'ORG': ['Майами', 'Лейкерс', 'Клипперс', 'Уор..."


In [98]:
c = Counter()
row_counter = df.Tags.apply(lambda x: Counter(x)).sum()

In [138]:
# Change number in first line to see named entities for more tags

for tag, _ in row_counter.most_common(1):
    df_tag  = df[df.Tags.apply(lambda x: True in pd.Series(x).isin([tag]).values)]
    l = []

    for x in df_tag.Named_entity:
        for v in x.values():
            for z in v:
                l.append(z)
    print(f'For tag {tag} 10 most common named entities are {[ ne[0] for ne in Counter(l).most_common(10)]}')

For tag Футбол 10 most common named entities are ['РПЛ', 'России', 'ЦСКА', 'Зенит', 'Спартак', 'Динамо', 'Зенита', 'Спартака', 'Sports.ru', 'Челси']
